# Plot heatmap data on world maps from *Natural Earth*

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext blackcellmagic

In [ ]:
import numpy as np
import pandas as pd

import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set_style("whitegrid", {"grid.color": "gainsboro"})

from tools import visualization

## Load trial data

In [ ]:
grouper_column_names = [
    "country_ISO",
    "country_continent",
    "subregion",
    "economy_level",
    "consolidated_economy_level",
    "income_group",
    "consolidated_income_group",
    "hdi_category",
]

In [ ]:
consolidated_counts_overall = {}
for grouper_column_name in grouper_column_names:
    consolidated_counts_overall[grouper_column_name] = pd.read_excel(
        "data/results/trials_sites_counts.xlsx",
        sheet_name="ovr_" + grouper_column_name,
    )

In [ ]:
consolidated_counts_overall["country_continent"]

In [ ]:
consolidated_counts_per_phase = {}
for grouper_column_name in grouper_column_names:
    consolidated_counts_per_phase[grouper_column_name] = pd.read_excel(
        "data/results/trials_sites_counts.xlsx",
        sheet_name="phs_" + grouper_column_name,
    )

In [ ]:
consolidated_counts_per_phase["country_continent"].head()

## Load and prepare geography data from naturalearth
We need the country outlines to generate choropleth maps; download from https://www.naturalearthdata.com/.

In [ ]:
geo_data = gpd.read_file("data/source/naturalearth/110m_cultural/ne_110m_admin_0_countries.shx")

### Remove Antarctica and Seven seas
We removed them from our original data set, and we omit them in the maps, too.

In [ ]:
geo_data = geo_data[
    ~geo_data["CONTINENT"].isin(["Antarctica", "Seven seas (open ocean)"])
].copy()

### Keep relevant columns and rename for convenience
Socioeconomic data is already in our master data set, we just need the ISO code for mapping and the geometry data.

In [ ]:
geo_data = (
    geo_data[
        [
            "ADM0_A3",
            "SUBREGION",
            "geometry",
        ]
    ]
    .rename(
        columns={
            "ADM0_A3": "country_ISO",
            "SUBREGION": "subregion",
        }
    )
    .copy()
)

In [ ]:
geo_data.head()

In [ ]:
len(geo_data)

### Plot a world map as sanity check

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 4))
geo_data.plot(edgecolor=u"white", color="silver", linewidth=0.25, ax=ax)
sns.despine(bottom=True, top=True, left=True, right=True)
fig.tight_layout()
ax.set_xlim((-185, 185))
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(False)
plt.show()

## Meta - plot population data on world map

### Population

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="population_estimate",
    geometry_base_dataframe=geo_data,
    log_scale=False,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"white",
    edges_linewidth=0.25,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=ax,
)
g.set_title("Population estimate from Natural Earth data")
fig.tight_layout()
plt.show()

### Log of population

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="population_estimate",
    geometry_base_dataframe=geo_data,
    log_scale=True,
    log_scale_diverging_palette=False,
    colormap_minimum_value=3,
    colormap_maximum_value=10,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Population estimate from Natural Earth data, log-scaled")
fig.tight_layout()
plt.show()

### Log of population, alternative colormap

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="population_estimate",
    geometry_base_dataframe=geo_data,
    log_scale=True,
    log_scale_diverging_palette=True,
    colormap_minimum_value=3,
    colormap_maximum_value=10,
    diverging_palette="RdBu",
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Population estimate from Natural Earth data, log-scaled")
fig.tight_layout()
plt.show()

## Plot overall trial counts

### Trials per country

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="n_trials",
    log_scale=False,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials in MS from clinicaltrials.gov, 1994 - 2023 - number of studies with at least 1 site in a given country")
fig.tight_layout()
plt.show()

### Trials per country per capita

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="trials_per_capita",
    log_scale=False,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials per capita in MS from clinicaltrials.gov, 1994 - 2023")
fig.tight_layout()
plt.show()

### Trials per country per capita, log

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="trials_per_capita",
    log_scale=True,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials per capita in MS from clinicaltrials.gov, 1994 - 2023, log-scaled")
fig.tight_layout()
plt.show()

### Trials per country per capita, log - alternative colormap

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="trials_per_capita",
    log_scale=True,
    log_scale_diverging_palette=True,
    diverging_palette="RdBu",
    colormap_minimum_value=None,
    colormap_maximum_value=None,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials per capita in MS from clinicaltrials.gov, 1994 - 2023, log-scaled")
fig.tight_layout()
plt.show()

### Log-disproportionality

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="factor_deviation_n_trials_from_expected",
    log_scale=True,
    log_scale_diverging_palette=True,
    colormap_minimum_value=-1,
    colormap_maximum_value=3,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Log-disproportionality of number of actual trials vs. expected number of trials based on population")
fig.tight_layout()
plt.show()

## Plot overall site counts

### Trial sites per country

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="n_sites",
    log_scale=False,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials in MS from clinicaltrials.gov, 1994 - 2023 - number of trial sites")
fig.tight_layout()
plt.show()

### Trial sites per country per capita

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="sites_per_capita",
    log_scale=False,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials in MS from clinicaltrials.gov, 1994 - 2023 - trial sites per capita")
fig.tight_layout()
plt.show()

### Trials per country per capita, log

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="sites_per_capita",
    log_scale=True,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials in MS from clinicaltrials.gov, 1994 - 2023 - trial sites per capita, log-scaled")
fig.tight_layout()
plt.show()

### Trials per country per capita, log - alternative colormap

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="sites_per_capita",
    log_scale=True,
    log_scale_diverging_palette=True,
    diverging_palette="RdBu",
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials in MS from clinicaltrials.gov, 1994 - 2023 - trial sites per capita, log-scaled")
fig.tight_layout()
plt.show()

### Log-disproportionality

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=consolidated_counts_overall["country_ISO"],
    column_to_plot="factor_deviation_n_sites_from_expected",
    log_scale=True,
    log_scale_diverging_palette=True,
    colormap_minimum_value=-2.5,
    colormap_maximum_value=1.5,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Log-disproportionality of actual number of trial sites vs. expected number of trial sites based on population")
fig.tight_layout()
plt.show()

## 4 x 4 plots with counts per phase

In [ ]:
phase_country_data = consolidated_counts_per_phase["country_ISO"]

### Trial sites per capita

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(24, 10))

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="sites_per_capita",
        log_scale=False,
        log_scale_diverging_palette=False,
        show_colorbar=False,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1] + ") Phase " + str(i) + ", trial sites per capita", fontsize=14
    )

fig.tight_layout()
plt.show()

### Trial sites per capita with standardized colormap

In [ ]:
min(phase_country_data["sites_per_capita"]), max(phase_country_data["sites_per_capita"])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(24, 10))

colormap_min = min(phase_country_data["sites_per_capita"])
colormap_max = max(phase_country_data["sites_per_capita"])

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="sites_per_capita",
        log_scale=False,
        log_scale_diverging_palette=False,
        colormap_minimum_value=colormap_min,
        colormap_maximum_value=colormap_max,
        show_colorbar=True,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1] + ") Phase " + str(i) + ", trial sites per capita", fontsize=14
    )

fig.tight_layout()
plt.show()

### Trial sites per capita, log

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(24, 10))

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="sites_per_capita",
        log_scale=True,
        log_scale_diverging_palette=False,
        show_colorbar=False,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1] + ") Phase " + str(i) + ", trial sites per capita, log-scaled",
        fontsize=14,
    )

fig.tight_layout()
plt.show()

### Trial sites per capita, log with normalized colormap

In [ ]:
colormap_min = min(
    np.log10(
        phase_country_data[phase_country_data["sites_per_capita"] > 0][
            "sites_per_capita"
        ]
    )
)
colormap_max = max(
    np.log10(
        phase_country_data[phase_country_data["sites_per_capita"] > 0][
            "sites_per_capita"
        ]
    )
)

fig, axes = plt.subplots(2, 2, figsize=(24, 10))

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="sites_per_capita",
        log_scale=True,
        log_scale_diverging_palette=False,
        colormap_minimum_value=colormap_min,
        colormap_maximum_value=colormap_max,
        show_colorbar=True,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1] + ") Phase " + str(i) + ", trial sites per capita, log-scaled",
        fontsize=14,
    )

fig.tight_layout()
plt.show()

### Log-disproportionality of trial sites

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(24, 10))

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="factor_deviation_n_sites_from_expected",
        log_scale=True,
        log_scale_diverging_palette=True,
        show_colorbar=False,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1]
        + ") Phase "
        + str(i)
        + ", log-disproportionality of actual number of trial sites vs. expected number of trial sites based on population",
        fontsize=14,
    )

fig.tight_layout()
plt.show()

### Log-disproportionality of trial sites - automatic colorbars

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(24, 10))

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="factor_deviation_n_sites_from_expected",
        log_scale=True,
        log_scale_diverging_palette=True,
        show_colorbar=True,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1]
        + ") Phase "
        + str(i)
        + ", log-disproportionality of actual number of trial sites vs. expected number of trial sites based on population",
        fontsize=14,
    )

fig.tight_layout()
plt.show()

### Log-disproportionality of trial sites - standardized colormap

In [ ]:
colormap_min = -2.5
colormap_max = 1.5

fig, axes = plt.subplots(2, 2, figsize=(24, 10))

for i in range(1, 5):

    visualization.plot_choropleth_map_country_level(
        trial_data=phase_country_data[phase_country_data["phase"] == "PHASE" + str(i)],
        column_to_plot="factor_deviation_n_sites_from_expected",
        log_scale=True,
        log_scale_diverging_palette=True,
        colormap_minimum_value=colormap_min,
        colormap_maximum_value=colormap_max,
        show_colorbar=True,
        edgecolor=u"black",
        edges_linewidth=0.25,
        geometry_base_dataframe=geo_data,
        geometry_column_name="geometry",
        country_id_column_name="country_ISO",
        base_color="silver",
        base_edgecolor=u"white",
        ax=axes[(i - 1) // 2][(i - 1) % 2],
    )
    axes[(i - 1) // 2][(i - 1) % 2].set_title(
        "ABCD"[i - 1]
        + ") Phase "
        + str(i)
        + ", log-disproportionality of actual number of trial sites vs. expected number of trial sites based on population",
        fontsize=14,
    )

fig.tight_layout()
plt.show()

## Plot regions instead of single countries

In [ ]:
regional_data_overall = consolidated_counts_overall["subregion"].copy()

In [ ]:
regional_data_overall.head()

In [ ]:
regional_data_overall = pd.merge(left=geo_data[["country_ISO", "subregion"]], right=regional_data_overall, on="subregion", how="left")

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    trial_data=regional_data_overall,
    column_to_plot="n_trials",
    log_scale=False,
    drop_zeros=False,
    log_scale_diverging_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Industry funded interventional drug trials in MS from clinicaltrials.gov, 1994 - 2023 - number of studies with at least 1 site in a given region")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

g = visualization.plot_choropleth_map_country_level(
    data=regional_data_overall,
    column_to_plot="sites_per_capita",
    log_scale=False,
    drop_zeros=False,
    log_scale_divergent_palette=False,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Trial sites per capita")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

colormap_min = -9
colormap_max = -4.5

g = visualization.plot_choropleth_map_country_level(
    trial_data=regional_data_overall,
    column_to_plot="sites_per_capita",
    log_scale=True,
    drop_zeros=True,
    log_scale_diverging_palette=False,
    colormap_minimum_value=colormap_min,
    colormap_maximum_value=colormap_max,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Trial sites per capita, log-scaled")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

colormap_min = -1.5
colormap_max = 2.5

g = visualization.plot_choropleth_map_country_level(
    trial_data=regional_data_overall,
    column_to_plot="factor_deviation_n_trials_from_expected",
    log_scale=True,
    drop_zeros=True,
    log_scale_diverging_palette=True,
    colormap_minimum_value=colormap_min,
    colormap_maximum_value=colormap_max,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Log-disproportionality of number of trials")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

colormap_min = -4
colormap_max = 1

g = visualization.plot_choropleth_map_country_level(
    trial_data=regional_data_overall,
    column_to_plot="factor_deviation_n_sites_from_expected",
    log_scale=True,
    drop_zeros=True,
    log_scale_diverging_palette=True,
    colormap_minimum_value=colormap_min,
    colormap_maximum_value=colormap_max,
    show_colorbar=True,
    edgecolor=u"black",
    edges_linewidth=0.25,
    geometry_base_dataframe=geo_data,
    geometry_column_name="geometry",
    country_id_column_name="country_ISO",
    base_color="silver",
    base_edgecolor=u"white",
    ax=None,
)
g.set_title("Log-disproportionality of number of trial sites")
fig.tight_layout()
plt.show()